In [1]:
from langchain_community.tools import WikipediaQueryRun

In [2]:
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max = 200)
wiki_tool = WikipediaQueryRun(api_wrapper = api_wrapper)

In [4]:
wiki_tool.name

'wikipedia'

In [5]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
llm = Ollama(model = "llama3.2")

/tmp/ipykernel_81428/1028341570.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "llama3.2")


In [7]:
loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
doc = loader.load()

In [8]:
document = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(doc)

In [9]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = [embedding_model.encode(doc.page_content) for doc in document]

In [10]:
texts = [doc.page_content for doc in document]

embeddings = embedding_model.encode(texts, convert_to_tensor=True)

In [11]:
class EmbeddingWrapper:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=True)


In [12]:
embedding_wrapper = EmbeddingWrapper(embedding_model)

In [13]:
texts = [doc.page_content for doc in document]

In [14]:
vectordb = FAISS.from_texts(texts, embedding_wrapper)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [15]:
retriever = vectordb.as_retriever()

In [16]:
retriever

VectorStoreRetriever(tags=['FAISS'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8769adb950>, search_kwargs={})

In [17]:
from langchain.tools.retriever import create_retriever_tool

In [18]:
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search information about langsmith on the internet.")

In [19]:
# arxiv
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [20]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [21]:
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))

In [22]:
arxiv.name

'arxiv'

In [23]:
tools = [wiki_tool, arxiv, retriever_tool]

In [24]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/root/anaconda3/envs/myenv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search information about langsmith on the internet.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f8888eb7600>, retriever=VectorStoreRetriever(tags=['FAISS'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8769adb950>, search_kwargs={}), 

In [25]:
from langchain import hub

In [29]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")


llm = Ollama(model= "llama3.2", temperature=0)

In [28]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("pisles/rag-prompt-llama-ps")
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="[INST]<<SYS>> You are an assistant for analyzing transcriptions from body-worn cameras, interviews, surveillance records, and other official content. Use the retrieved context to provide analysis for officials working with federal and state agencies, such as police departments or investigators. Answer concisely within three sentences. If the answer is unclear, state that you don't know.<</SYS>> \nQuestion: {question} \nContext: {context} \nAnswer: [/INST]"), additional_kwargs={})]

In [38]:
from langchain.agents import AgentExecutor
import requests

In [50]:
def query_llama(prompt, model="llama-3.2", temperature=0):
    api_key = os.getenv('OLLAMA_API_KEY')
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    data = {
        'model': model,
        'messages': prompt.get_messages(),
        'temperature': temperature
    }

    # Debug: Print the data structure
    print("Data being sent to API:", data)

    response = requests.post('https://api.ollama.com/v1/chat/completions', headers=headers, json=data)
    return response.json()

In [51]:
# Define a custom prompt class
class CustomPrompt:
    def __init__(self):
        self.messages = []

    def add_message(self, role, content):
        # Ensure that both role and content are strings
        self.messages.append({"role": str(role), "content": str(content)})

    def get_messages(self):
        # Return a list of dictionaries with simple data types
        return self.messages

In [52]:
# Define a custom agent class
class LlamaAgent:
    def __init__(self, model, tools, prompt):
        self.model = model
        self.tools = tools
        self.prompt = prompt

    def run(self, user_input):
        # Add user input to the prompt
        self.prompt.add_message("user", user_input)

        # Query the LLaMA model
        response = query_llama(self.prompt, model=self.model)

        # Process the response and use tools if necessary
        # This is where you would integrate any additional logic or tools
        return response

In [53]:
prompt = CustomPrompt()
prompt.add_message("system", "You are a helpful assistant.")
tools = [wiki_tool, arxiv, retriever_tool]

In [56]:
agent = LlamaAgent(model="llama3.2", tools=tools, prompt=prompt)
response = agent.run("Tell me about Langsmith")
print(response)

Data being sent to API: {'model': 'llama3.2', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tell me about Langsmith'}, {'role': 'user', 'content': 'Tell me about Langsmith'}, {'role': 'user', 'content': 'Tell me about Langsmith'}], 'temperature': 0}


ConnectionError: HTTPSConnectionPool(host='api.ollama.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f885d51ec00>: Failed to resolve 'api.ollama.com' ([Errno -5] No address associated with hostname)"))